In [125]:
import pandas as pd
from fbprophet import Prophet
import plotly.express as px
import plotly.io as pio
import base64
import yfinance as yf
import pandas_datareader as pdr
import datetime as dt
from datetime import date
import os
from PIL import Image

In [126]:
tech_stocks = ["FB", "AAPL", "AMZN", "TSLA", "GOOG", "NVDA", "NFLX"]
dfs = list()
prop_dfs = list()

In [127]:
for i in tech_stocks:
    yf.pdr_override()
    today_date = date.today()
    dfs.append(pdr.get_data_yahoo(i, start = today_date - dt.timedelta(days = 5 * 365), end = today_date))

In [128]:
def initializer(df, c):
    if 'Date' not in df.columns:
        df = df.reset_index()
        
    if not os.path.isdir("output/" + str(tech_stocks[c]) + "/files"):
        os.makedirs("output/" + str(tech_stocks[c]) + "/files")
        
    f = open("output/" + tech_stocks[c] + "/files/analysis.txt" , "w")
    #f = open("files/file" + str((c % len(tech_stocks)) + 1) + ".txt" , "w")
    f.write("Stock Information for Ticker: " + str(tech_stocks[c]) + "\n")
    f.write(df.mean().to_string())
    f.write("\n\n\n")
    f.write(df.describe().to_string())
    f.close()
    
    return df

In [129]:
def visualize(df, c):
    width = 600
    height = 600
    figs = list()
    figs.append(px.area(df, x = "Date", y = "Close"))
    figs.append(px.line(df, x = "Date", y = "Close"))
    figs.append(px.area(df, x = "Date", y = "Volume"))
    figs.append(px.line(df, x = "Date", y = "Volume"))   
    figs.append(px.bar(df, y = "Volume"))
    figs.append(px.box(df, y = "Close"))

    if not os.path.isdir("output/" + str(tech_stocks[c]) + "/graphs"):
        os.makedirs("output/" + str(tech_stocks[c]) + "/graphs")
        
    
    for i, j in enumerate(figs):
        pio.write_image(j, "output/" + str(tech_stocks[c]) + "/graphs/occurred" + str(i+1) + ".png")

In [130]:
def preprocess(df):
    columns = ["Date", "Close"]
    prop_df = pd.DataFrame(df, columns = columns)
    prop_df = prop_df.rename(columns = {"Date": "ds", "Close": "y"})
    return prop_df

In [131]:
def forecaster(prop_df, c):
    m = Prophet()
    m.fit(prop_df)
    
    future = m.make_future_dataframe(periods = 36, freq = "M")
    forecast = m.predict(future)
    
    if not os.path.isdir("output/" + str(tech_stocks[c]) + "/files"):
        os.makedirs("output/" + str(tech_stocks[c]) + "/files")
        
    
    f = open("output/" + tech_stocks[c] + "/files/analysis.txt" , "a")
    f.write("\n\n\n")
    f.write(forecast.to_string())
    f.close()
    #pred_visualization(forecast, m, c)

In [132]:
def pred_visualization(fc, m, c):
    figs = list()
    figs.append(px.line(fc, x = "ds", y = "yhat"))
    figs.append(m.plot(fc, xlabel = "ds", ylabel = "y"))
    figs.append(m.plot_components(fc))
    
    if not os.path.isdir("output/" + str(tech_stocks[c-1]) + "/graphs"):
        os.makedirs("output/" + str(tech_stocks[c-1]) + "/graphs")
    
    for i, j in enumerate(figs):
        pio.write_image(j, "output/" + str(tech_stocks[c-1]) + "/graphs/forecast" + str(i+1) + ".png")

In [133]:
for i in range(len(dfs)):
    dfs[i] = initializer(dfs[i], i)
    visualize(dfs[i], i)
    prop_dfs.append(preprocess(dfs[i]))
    forecaster(prop_dfs[i], i)

/var/folders/s1/y8hjc_rd18g3zw5xh5l4_y8r0000gn/T/ipykernel_70136/3295268458.py:11: FutureWarning:

DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


Initial log joint probability = -19.8479
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       3612.97     0.0375488       408.098           1           1      117   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     199       3645.28    0.00207562       530.078      0.7274      0.7274      227   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     299       3659.88    0.00644323       151.611           1           1      340   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     343       3662.17   0.000282054        553.06   4.231e-07       0.001      445  LS failed, Hessian reset 
     399       3664.76   0.000528761       238.082      0.4113           1      516   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     499       3670.41     0.0011138    

/var/folders/s1/y8hjc_rd18g3zw5xh5l4_y8r0000gn/T/ipykernel_70136/3295268458.py:11: FutureWarning:

DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


Initial log joint probability = -23.1983
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       3633.04     0.0140845        455.38           1           1      123   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     199       3731.96   0.000866115       161.334      0.4005           1      241   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     299       3740.39    0.00661159       169.853           1           1      366   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     324       3742.63   0.000211344       407.236   6.218e-07       0.001      434  LS failed, Hessian reset 
     399       3746.81     0.0128085       389.789      0.6847           1      521   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     499       3752.72    0.00422445    

/var/folders/s1/y8hjc_rd18g3zw5xh5l4_y8r0000gn/T/ipykernel_70136/3295268458.py:11: FutureWarning:

DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


...

Initial log joint probability = -29.0914
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       4048.77     0.0126557       3501.97           1           1      127   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     199        4123.1     0.0269314       1009.73           1           1      238   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     299       4162.28     0.0590864        1526.4           1           1      349   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     399       4191.71     0.0127227       808.664           1           1      468   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     499       4211.83     0.0374252       1494.22           1           1      577   
    Iter      log prob        ||dx||      ||grad||       alpha  

/var/folders/s1/y8hjc_rd18g3zw5xh5l4_y8r0000gn/T/ipykernel_70136/3295268458.py:11: FutureWarning:

DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


Initial log joint probability = -42.3179
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       3641.34    0.00829918       392.861           1           1      120   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     199       3698.44   0.000474957       758.255       0.579       0.579      228   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     299       3730.94    0.00916728       234.955           1           1      340   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     399       3763.26     0.0432795        462.57           1           1      455   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     499        3777.2     0.0258772       908.784           1           1      566   
    Iter      log prob        ||dx||      ||grad||       alpha  

/var/folders/s1/y8hjc_rd18g3zw5xh5l4_y8r0000gn/T/ipykernel_70136/3295268458.py:11: FutureWarning:

DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


Initial log joint probability = -7.13672
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       3338.33    0.00442434        502.63      0.6962      0.6962      117   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     199       3382.69    0.00640728       245.569           1           1      235   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     299       3415.96    0.00205159       198.269           1           1      359   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     399        3434.6    0.00316859       231.968           1           1      478   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     499       3439.26    0.00166371       111.424           1           1      595   
    Iter      log prob        ||dx||      ||grad||       alpha  